<a href="https://colab.research.google.com/github/JunlinH/Space-Pathfinder/blob/master/song_predict_20_highlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
import numpy as np
import tensorflow.keras as keras
import music21
import pickle
import matplotlib.pyplot as plt
from music21 import converter, instrument, note, chord, midi, stream
import tensorflow as tf
from fractions import Fraction

In [24]:
def save_data(data, path):
    with open(path, 'wb') as file:
        pickle.dump(data, file)

def read_data(path):
    with open(path, 'rb') as file:
        data = pickle.load(file)
    return data

In [28]:
def predict_notes(notes_classes, model):
    index_note_dictionary = read_data("/content/gdrive/My Drive/Song/index_note_dictionary_20_highlow.pkl")
    X_estimate = read_data("/content/gdrive/My Drive/Song/X_estimate_20_highlow.pkl")
    seed_number = np.random.randint(0, len(X_estimate)-1)
    seed_notes = X_estimate[seed_number][0]
    predicted_ouput = []
    predicted_note = []
    predicted = []

    # generate 500 notes
    for i in range(260):
          seed_notes_reshape = np.reshape(seed_notes, (1, len(seed_notes), 1))
          seed_notes_normalized = seed_notes_reshape / float(notes_classes)
          # print(type(seed_notes_normalized))
          predicted_note_index = np.argmax(model.predict(seed_notes_normalized), axis=-1)[0,]
          predicted_note = index_note_dictionary[predicted_note_index]
          predicted_ouput.append(predicted_note)
          seed_notes.append(int(predicted_note_index))
          seed_notes = seed_notes[1:]
          if i == 79:
              seed_notes = X_estimate[seed_number][4]
          if i == 159:
              seed_notes = X_estimate[seed_number][7]

    # append with the X_estimate to form a song
    predicted += [index_note_dictionary[index] for index in X_estimate[seed_number][0]]
    predicted += predicted_ouput
    return predicted

In [26]:
def decode_output(predicted):
    output_notes = []
    notes_offset = 0.0
    midi_stream = stream.Stream()

    for pattern in predicted:
        #chord
        if ';' in pattern:
            notes_in_chord = pattern.split(';')
            notes_chord = []
            for i in range(len(notes_in_chord)-1):
                note_name = notes_in_chord[i]
                note_duration = int(Fraction(notes_in_chord[-1]))
                new_note = note.Note(note_name, quarterLength = note_duration)
                new_note.storedInstrument = instrument.Piano()
                notes_chord.append(new_note)
            new_chord = chord.Chord(notes_chord)
            output_notes.append(new_chord)
            midi_stream.insert(notes_offset, new_chord)
            if len(output_notes) > 1:
                if not isinstance(output_notes[-2], note.Note):
                    notes_offset += float(Fraction(notes_in_chord[-1]))
        #simulnatenous notes
        elif '*' in pattern:
            notes_in_simul = pattern.split('*')
            for i in range(len(notes_in_simul)-1):
                note_name_simul = notes_in_simul[i]
                note_duration_simul = int(Fraction(notes_in_simul[-1]))
                new_note_simul = note.Note(note_name_simul, quarterLength = note_duration_simul, offset = notes_offset)
                new_note_simul.storedInstrument = instrument.Piano()
                output_notes.append(new_note_simul)
                midi_stream.insert(notes_offset, new_note_simul)
            notes_offset += float(Fraction(notes_in_simul[-1]))
        #single note
        elif ' ' in pattern:
            single_note = pattern.split()
            note_name_single = single_note[0]
            note_duration_single = int(Fraction(single_note[-1]))
            new_note_single = note.Note(note_name_single, quarterLength = note_duration_single, offset = notes_offset)
            new_note_single.storedInstrument = instrument.Piano()
            output_notes.append(new_note_single)
            midi_stream.insert(notes_offset, new_note_single)
            notes_offset += float(Fraction(single_note[-1]))
    
    print(output_notes)
    print(notes_offset)

    midi_file = midi_stream.write('midi', fp='/content/gdrive/My Drive/Song/test_output_20_highlow_enhance.mid')

    return midi_file


In [29]:
if __name__ == '__main__':

    #load the model
    model = keras.models.load_model('/content/gdrive/My Drive/Song/song_model_20_highlow_enhance.h5')
    model.summary()
    #read data
    notes_classes = read_data('/content/gdrive/My Drive/Song/notes_classes_20_highlow.pkl')

    #predict the output
    predicted = predict_notes(notes_classes, model)
    print(predicted)

    #transform the output back into music notes
    midi_file = decode_output(predicted)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 20, 1024)          2105344   
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 1024)          4096      
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dense (Dense)                (None, 8419)              4318947   
_________________________________________________________________
dense_1 (Dense)              (None, 16838)             141775960 
Total params: 151,352,123
Trainable params: 151,350,075
Non-trainable params: 2,048
_________________________________________________________________
['E-2*F4*0.5', 'B-2*G3*0.5', 'E-3*B-3*0.5', 'F4 1.0', 'G3 0.5', 'E-2*B-3*0.5', 'F4 1.0', 'F2 0.5', 'C3*G